# The Internal Displacement Monitoring Centre (IDMC)
So until IDMC reply to me by email, we can use this dataset that i found on the internet. It is still IDMC, just processed by another 3rd party, which is not great but I think we can work with it. I created a small script that allows you to query on the country, date, hazard (category, and specific). The last parameter for the query i thought was particularly interesting, because maybe we will find different patterns within different hazards.
For hazards, they have three groupings, depending on the division. Here are some examples:
- hazard category (Weather related, Geophysical)
- hazard_sub_category_name (Hydrological, Climatological, Meteorological)
- hazard_type_name (Flood, Storm, Wildfire, Flood, Extreme Temperature, Volcanic activity)

I think this is a pretty good dataset with data from 2008 up until end of 2024. One downside of this dataset is that it doesnt have more accurate data regarding space, only country level. 

One more thing, i think when merging different datasets, we should do it on ISO3, then, to get the country name we should just map it with some library, this way, we will not have to deal with duplicates of countries.

In [4]:
import pandas as pd
from typing import Optional, Union, Iterable
from datetime import date


class DisasterDisplacementData:
    def __init__(self, csv_path: str):
        self.df = pd.read_csv(csv_path)
        self.df["start_date"] = pd.to_datetime(
            self.df["start_date"],
            format="%Y-%m-%d",
            errors="coerce"
        )
        
        self.df["end_date"] = pd.to_datetime(
            self.df["end_date"],
            format="%Y-%m-%d",
            errors="coerce"
        )

        for col in ["iso3", "hazard_category_name", "hazard_type_name"]:
            if col in self.df.columns:
                self.df[col] = self.df[col].astype(str).str.strip()

    def query(
        self,
        iso3: Optional[Union[str, Iterable[str]]] = None,
        start_date: Optional[Union[str, date]] = None,
        start_date_from: Optional[Union[str, date]] = None,
        start_date_to: Optional[Union[str, date]] = None,
        hazard_category_name: Optional[str] = None,
        hazard_type_name: Optional[str] = None,
    ) -> pd.DataFrame:

        result = self.df.copy()

        # ---- ISO3: single or multiple ----
        if iso3:
            if isinstance(iso3, str):
                iso3 = [iso3]
            iso3 = [code.strip() for code in iso3]
            result = result[result["iso3"].isin(iso3)]

        if hazard_category_name:
            result = result[
                result["hazard_category_name"].str.lower()
                == hazard_category_name.lower()
            ]

        if hazard_type_name:
            result = result[
                result["hazard_type_name"].str.lower()
                == hazard_type_name.lower()
            ]

        if start_date:
            start_date = pd.to_datetime(start_date)
            result = result[result["start_date"] == start_date]

        if start_date_from:
            start_date_from = pd.to_datetime(start_date_from)
            result = result[result["start_date"] >= start_date_from]

        if start_date_to:
            start_date_to = pd.to_datetime(start_date_to)
            result = result[result["start_date"] <= start_date_to]

        return result.reset_index(drop=True)


In [5]:
idmp_database = DisasterDisplacementData("internal-displacements-new-displacements-associated-with-disasters.csv")
idmp_database.query(
    iso3=["PHL", "IDN"],
    hazard_type_name="Flood",
    start_date_from="2008-01-01",
    start_date_to="2008-12-31"
)

,iso3,country_name,year,start_date,start_date_accuracy,end_date,end_date_accuracy,event_name,hazard_category,hazard_category_name,...,hazard_sub_category_name,hazard_type,hazard_type_name,hazard_sub_type,hazard_subtype_name,new_displacement,new_displacement_rounded,total_displacement,total_displacement_rounded,event_codes
0,IDN,Indonesia,2008,2008-01-02,Day,2008-02-06,Day,Indonesia: Flood - 02/01/2008,2,Weather related,...,Hydrological,10,Flood,13,NaN,49653,50000.0,NaN,NaN,[]
1,PHL,Philippines,2008,2008-01-12,Day,2008-12-04,Day,Philippines: Flood - 12/01/2008,2,Weather related,...,Hydrological,10,Flood,13,NaN,51222,51000.0,NaN,NaN,[]
2,IDN,Indonesia,2008,2008-01-30,Day,2008-01-31,Day,Indonesia: Flood - 30/01/2008,2,Weather related,...,Hydrological,10,Flood,13,NaN,40000,40000.0,NaN,NaN,[]
3,PHL,Philippines,2008,2008-04-09,Day,2008-09-13,Day,Philippines: Flood - 09/04/2008,2,Weather related,...,Hydrological,10,Flood,13,NaN,350000,350000.0,NaN,NaN,[]
4,IDN,Indonesia,2008,2008-04-23,Day,2008-04-27,Day,Indonesia: Flood - 23/04/2008,2,Weather related,...,Hydrological,10,Flood,13,NaN,34000,34000.0,NaN,NaN,[]
5,IDN,Indonesia,2008,2008-06-09,Day,2008-09-08,Day,Indonesia: Flood - 09/06/2008,2,Weather related,...,Hydrological,10,Flood,13,NaN,17352,17000.0,NaN,NaN,[]
6,IDN,Indonesia,2008,2008-10-01,Month,2008-10-01,Month,Indonesia: Flood - 01/10/2008,2,Weather related,...,Hydrological,10,Flood,13,NaN,31500,32000.0,NaN,NaN,[]
7,IDN,Indonesia,2008,2008-10-03,Day,2008-04-03,Day,Indonesia: Flood - 03/10/2008,2,Weather related,...,Hydrological,10,Flood,13,NaN,5060,5100.0,NaN,NaN,[]
8,IDN,Indonesia,2008,2008-11-02,Day,2008-02-27,Day,Indonesia: Flood - 02/11/2008,2,Weather related,...,Hydrological,10,Flood,13,NaN,3500,3500.0,NaN,NaN,[]
9,IDN,Indonesia,2008,2008-11-11,Day,2008-11-16,Day,Indonesia: Flood - 11/11/2008,2,Weather related,...,Hydrological,10,Flood,13,NaN,2400,2400.0,NaN,NaN,[]
